# Imports

In [1]:
# import required libraries

# data manipulation
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# path
from pathlib import Path

# date operations
from datetime import datetime, timedelta

# supress non-critical warnings
import warnings
warnings.filterwarnings('ignore')

# set pandas options
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [2]:
# construct project root path
# update this with your own project root path, to run the notebook seamlessly!
project_root = Path("/Users/saibharathbesta/projects/data_analytics_assignment_1/")

# Read the data

In [3]:
# load the data 

# construct the path of the file
file_path = project_root / "data/M25_DA_A1_Dataset2.csv"

# read the csv file from file_path using pandas
df = pd.read_csv(file_path)

In [4]:
df.head()

,ID,Possession Status,Availability Starts From,Floor No,Commercial,Developer,Approved Authority Name,Units Available,Price,Price (English),Flooring Type,Electricity Status,Maintenance Type,Maintenance Charges,Booking Amount,Landmark,Covered Area,Project Name,sqft Price,Carpet Area,Area Name,Property Uniqueness,Unit of Carpet Area,Society,Ownership Type,furnished Type,Bathroom,Parking,Facing,Amenities Facing,Transaction Type,Floor Data,Type of Property,City,bedroom,Data Referred From,floors,Water Status,Luxury Flat,Covered Area.1,Carpet Area Unit,isPrimeLocationProperty,Tenants Preference,Location,Property,covArea Unit,isLuxuryServiceProvided,Property Lifespan,balconies,Rera,Power Back Up,Lift,Rain Water Harvesting,Club House,Swimming Pool,Gymnasium,Park,Parking.1,Security,Water Storage,Private Terrace/Garden,Vaastu Compliant,Service/Goods Lift,Air Conditioned,Visitor Parking,Intercom Facility,Maintenance Staff,Waste Disposal,Laundry Service,Internet/Wi-Fi Connectivity,DTH Television Facility,RO Water System,Banquet Hall,Bar/Lounge,Cafeteria/Food Court,Conference Room,Piped Gas,Jogging and Strolling Track,Outdoor Tennis Courts,Mansion,Downtown,Skyline View,Wrap Around Balcony,Island Kitchen Layout,Full Glass Wall,House help accommodation,Concierge Services,Garden View,Marble flooring,Well Furnished,Modular Kitchen,Helipad,Private pool,Private Jaccuzi,Mini Cinema Theatre,Golf Course,Infinity Swimming Pool,Pool with temperature control,Sea facing,Skydeck,Earth quake resistant,Theme based Architectures,Health club with Steam / Jaccuzi,Large Clubhouse,Large Open space,Fingerprint Access,Grand Entrance lobby,Private Garage,Fireplace,Wine Cellar,Sky Villa,Water Front,Hilltop,Smart Home,Barbeque space,Kids play area,Library,Puja Room,Study,Servant Room,Store,Aerobics Room,Canopy Walk,Coffee Lounge & Restaurants,Dance Studio,Event Space & Amphitheatre,Flower Gardens,Guest Accommodation,Indoor Squash & Badminton Courts,Kids Play Area,Kids Play Pool With Water Slides,Library And Business Centre,Meditation Area,Multipurpose Courts,Recreational Pool,Rentable CommuniPfty Space,Retail Boulevard (Retail Shops),Cycling & Jogging Track,Fire Fighting Equipment,Aesthetically designed landscape garden,Possession By,NRI Pref,Land Area / Covered Area,Land Area Unit,Pantry Type
0,12685,Under Construction,Dec '25,5,N,NaN,KDMC,1.0,3150000.0,31.5 Lac,Vitrified,No/Rare Powercut,Per sq. Unit Monthly,3.0,100000.0,Kalyan West Bapgaon,635.0,NaN,4960.0,375.0,Kalyan West,NaN,Sq-ft,Y,Freehold,Unfurnished,2.0,1 Covered,East,"Garden/Park, Pool",New Property,5 of 14 Floor,Apartment,Thane,1,Agent,14.0,Water Availability 24 Hours Available,F,635,Sq-ft,Y,Bachelors/Family,"Kalyan West, Thane",1 BHK Flat,Sq-ft,0,NaN,2.0,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN
1,12684,Ready to Move,NaN,20,Y,TATA Housing Development Company Ltd.,TMC,10.0,6300000.0,63 Lac,Vitrified,No/Rare Powercut,Per sq. Unit Monthly,3.0,100000.0,Rajoli naka,579.0,Tata Amantra,10880.0,579.0,Kalyan West,NaN,Sq-ft,Y,Freehold,Unfurnished,2.0,1 Covered,East,Main Road,New Property,20 of 34 Floor,Apartment,Thane,2,Agent,34.0,Water Availability 24 Hours Available,F,579,Sq-ft,Y,Bachelors/Family,"Kalyan West, Thane",2 BHK Flat,Sq-ft,0,New Construction,NaN,Y,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN
2,12683,Ready to Move,NaN,18,N,Sai Satyam Developers,KDMC,0.0,5400000.0,54 Lac,Vitrified,No/Rare Powercut,Monthly,1200.0,100000.0,This property has reputed scholl ints vicinity.,850.0,Sai Satyam Homes,6352.0,585.0,Kalyan West,East Facing Property,Sq-ft,Y,Freehold,Unfurnished,2.0,1 Open,East,"Garden/Park, Main Road",New Property,18 of 18 Floor,Apartment,Thane,2,Agent,18.0,Water Availability 24 Hours Ava

# Let's understand the dataset before starting!

In [5]:
# understanding the size of the dataset
df.shape

(12685, 145)

In [6]:
# convert all columns to numeric
# if a column contains non-numeric values, those values will be ignored
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

In [7]:
# check datatypes of each feature
df.dtypes

ID                                           int64
Possession Status                           object
Availability Starts From                    object
Floor No                                    object
Commercial                                  object
Developer                                   object
Approved Authority Name                     object
Units Available                            float64
Price                                      float64
Price (English)                             object
Flooring Type                               object
Electricity Status                          object
Maintenance Type                            object
Maintenance Charges                        float64
Booking Amount                             float64
Landmark                                    object
Covered Area                               float64
Project Name                                object
sqft Price                                 float64
Carpet Area                    

In [8]:
# check for null values
df.isnull().sum()

ID                                             0
Possession Status                             78
Availability Starts From                    9602
Floor No                                     858
Commercial                                     0
Developer                                   5679
Approved Authority Name                     6542
Units Available                             6550
Price                                         84
Price (English)                                0
Flooring Type                               4534
Electricity Status                          5425
Maintenance Type                            5613
Maintenance Charges                         6402
Booking Amount                              4984
Landmark                                    4644
Covered Area                                1311
Project Name                                4531
sqft Price                                  1205
Carpet Area                                 2552
Area Name           

In [9]:
# drop duplicate rows from the dataframe
df = df.drop_duplicates()

In [10]:
# take a look at the shape again
df.shape

(12685, 145)

# 1. Price Segmentation & Market Overview [7.5 Marks]

In [11]:
# create price segments

# calculate percentiles for price segmentation
low_threshold = df['Price'].quantile(0.33)
high_threshold = df['Price'].quantile(0.67)

print("")

# create price segments
def categorize_price(price: float, low_threshold: float = low_threshold, high_threshold: float = high_threshold) -> str:
    """
    create three price segments based on data distribution

    Args:
        price (float): price of the property

    Returns:
        str: segment based on threshold
    """
    if price <= low_threshold:
        return 'Low-Budget'
    elif price <= high_threshold:
        return 'Medium-Budget'
    else:
        return 'Premium'

df['Price_Segment'] = df['Price'].apply(categorize_price)

# print thresholds
print(f"Price Segmentation Thresholds:")
print(f"Low-Budget (Low): Rs.0 - Rs.{low_threshold}")
print(f"Medium-Budget: Rs.{low_threshold} - Rs.{high_threshold}")
print(f"Premium: Rs.{high_threshold}+")


Price Segmentation Thresholds:
Low-Budget (Low): Rs.0 - Rs.8700000.0
Medium-Budget: Rs.8700000.0 - Rs.20000000.0
Premium: Rs.20000000.0+


### Visualisations for price analysis

In [12]:
# create subplot figure
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Price Segment Distribution', 
        'Properties by City & Price Segment',
        'Property Types by Price Segment', 
        'Average Price by City',
        'Bedroom Distribution by Segment', 
        'Price Range Analysis'
    ),
    specs=[[{"type": "pie"}, {"type": "bar"}],
            [{"type": "bar"}, {"type": "bar"}],
            [{"type": "bar"}, {"type": "box"}]]
)

In [13]:
# 1. price segment distribution (Pie Chart)
segment_counts = df['Price_Segment'].value_counts()

fig.add_trace(
    go.Pie(
        labels=segment_counts.index, 
        values=segment_counts.values,
        name="Price Segments", 
        hole=0.4,
        showlegend=False,
    ),
    row=1, col=1
)

# 2. properties by City & Price Segment
city_segment = df.groupby(['City', 'Price_Segment']).size().unstack(fill_value=0)

for i, segment in enumerate(city_segment.columns):
    fig.add_trace(
        go.Bar(
            name=segment, 
            x=city_segment.index, 
            y=city_segment[segment],
            showlegend=False,
        ),
        row=1, col=2
    )

# 3. property Types by Price Segment
prop_segment = df.groupby(['Type of Property', 'Price_Segment']).size().unstack(fill_value=0)

for i, segment in enumerate(prop_segment.columns):
    fig.add_trace(
        go.Bar(
            name=segment, 
            x=prop_segment.index, 
            y=prop_segment[segment],
            showlegend=False,
        ),
        row=2, col=1
    )

# 4. average Price by City
avg_price_city = df.groupby('City')['Price'].mean().sort_values(ascending=False)

fig.add_trace(
    go.Bar(
        x=avg_price_city.index,
        y=avg_price_city.values,
        name='Avg Price',
        showlegend=False,
    ),
    row=2, col=2
)

# 5. bedroom distribution by Segment
bedroom_segment = df.groupby(['Price_Segment', 'bedroom']).size().unstack(fill_value=0)

for bedroom in bedroom_segment.columns:
    fig.add_trace(
        go.Bar(
            name=f'{bedroom} BHK', 
            x=bedroom_segment.index, 
            y=bedroom_segment[bedroom],
            showlegend=False
        ),
        row=3, col=1
    )

# 6. Price range analysis (Box Plot)
for i, segment in enumerate(df['Price_Segment'].unique()):
    segment_data = df[df['Price_Segment'] == segment]
    fig.add_trace(
        go.Box(
            y=segment_data['Price'],
            name=segment,
            showlegend=False,
        ),
        row=3, col=2
    )


# Update layout
fig.update_layout(
title_text="Price Segmentation Analysis & Market Overview",
height=1200,
showlegend=True,
title_x=0.5,
font=dict(size=12)
)
# Update y-axes labels
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_yaxes(title_text="Count", row=2, col=1)
fig.update_yaxes(title_text="Average Price (Rs.)", row=2, col=2)
fig.update_yaxes(title_text="Count", row=3, col=1)
fig.update_yaxes(title_text="Price (Rs.)", row=3, col=2)



### Visualisations for to create a summary of amenities

In [14]:
# let's derive all the amenities from the dataset

def get_amenities_list(df: pd.DataFrame) -> list[str]:
    """
    create a list of amenities based on a business logic acquired during initial analysis.

    Args:
        df (pd.DataFrame): input dataframe

    Returns:
        list[str]: list of all amenities
    """
    # initialise an empty amenity list
    amenities_list = []
    # create a dataframe with datatypes
    df_dtypes = pd.DataFrame(df.dtypes).reset_index()
    df_dtypes.columns = ["col_name", "datatype"]

    # get all the integer columns
    int_cols = df_dtypes[df_dtypes["datatype"] == "int64"]["col_name"].tolist() 
    # loop through all integer columns and find where only [0,1] are only possible values
    # if yes then consider it an amenity for now!
    for col in int_cols:
        if sorted(df[col].unique().tolist()) == [0, 1]:
            amenities_list.append(col)
    
    return amenities_list

# create a list of amenities
amenities_list = get_amenities_list(df)

In [15]:
# analyze key amenities across price segments!!
# to analyze the key amenities, we will calculate the presence of each amenity across various price segment!!
# we will further create a heatmap of this data!

# initialise a dictionary to store amenity data!!
amenity_analysis = {}

# loop over each price segment
for segment in df['Price_Segment'].unique():
    segment_data = df[df['Price_Segment'] == segment]
    total_properties = len(segment_data)
    
    amenity_stats = {}
    # loop over each amenity
    for amenity in amenities_list:
        # check the data type of the amenity column
        if segment_data[amenity].dtype == 'object':
            # count 'Y', 'Yes', '1' as positive
            positive_count = segment_data[amenity].isin(['Y', 'Yes', '1', 1]).sum()
        else:
            positive_count = (segment_data[amenity] == 1).sum()
        
        # calculate the percentage of positive count
        percentage = (positive_count / total_properties) * 100 if total_properties > 0 else 0
        amenity_stats[amenity] = percentage
    
    # add the analysis for the current segment to the dictionary
    amenity_analysis[segment] = amenity_stats

In [16]:
# prepare data for heatmap
# get the segments from the amenity analysis dictionary
segments = list(amenity_analysis.keys())

# create a list of lists, where each inner list contains the availability of amenities for a specific segment
z_data = []
for segment in segments:
    z_data.append([amenity_analysis[segment][amenity] for amenity in amenities_list])

# create the heatmap using plotly.graph_objects.Heatmap
fig = go.Figure(data=go.Heatmap(
    z=z_data,
    x=amenities_list,
    y=segments,
    colorscale='RdYlBu',  # use a color scale that starts with red (low availability) and ends with blue (high availability)
    text=[[f'{round(val,2)}%' for val in row] for row in z_data],  # add availability percentage to each cell
))

# update the layout of the figure
fig.update_layout(
    title='Amenities Availability Across Price Segments',  # set the title of the figure
    xaxis_title='Amenities',  # set the x-axis title
    yaxis_title='Price Segments',  # set the y-axis title
    height=400,  # set the height of the figure to 400
    title_x=0.5  # center the title horizontally
)


# 2. City-Level Comparative Analysis (Mumbai vs Thane) [7.5 Marks]

In [17]:
# filter for Mumbai and Thane only
city_filter = df['City'].isin(['Mumbai', 'Thane'])
df_filtered = df[city_filter].copy()

# create a new column called Property_Segment to standardize the data
# by default the property will be considered "Residential"
df_filtered['Commercial'] = df_filtered['Commercial'].fillna('N')
df_filtered['Property_Segment'] = df_filtered['Commercial'].apply(
    lambda x: 'Commercial' if x in ['Y', 'Yes', '1', 1] else 'Residential'
)

# calculate price per sqft foe each property
df_filtered['Price_Per_Sqft'] = np.where(
    df_filtered['Carpet Area'] > 0,
    round(df_filtered['Price'] / df_filtered['Carpet Area'], 2),
    df_filtered.get('sqft Price ', 0)
)

# print summary!
print(f"Data loaded successfully:")
print(f"Mumbai properties: {len(df_filtered[df_filtered['City'] == 'Mumbai'])}")
print(f"Thane properties: {len(df_filtered[df_filtered['City'] == 'Thane'])}")
print(f"Total properties for analysis: {len(df_filtered)}")

Data loaded successfully:
Mumbai properties: 8542
Thane properties: 4113
Total properties for analysis: 12655


In [ ]:
# create subplot figure
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Property Count by City & Segment',
        'Average Price Comparison',
        'Property Types Distribution',
        'Bedroom Configuration',
        'Price Distribution',
        'Price per Sqft Comparison'
    ),
    specs=[[{"type": "bar"}, {"type": "bar"}],
            [{"type": "bar"}, {"type": "bar"}],
            [{"type": "violin"}, {"type": "bar"}]]
)

# 1. property count by city and segment
segment_city = df_filtered.groupby(['City', 'Property_Segment']).size().unstack(fill_value=0)

for segment in segment_city.columns:
    fig.add_trace(
        go.Bar(
            name=segment,
            x=segment_city.index,
            y=segment_city[segment],
            text=segment_city[segment],
            textposition='auto',
            showlegend=False
        ),
        row=1, col=1
    )

# 2. average price comparison
avg_prices = df_filtered.groupby(['City', 'Property_Segment'])['Price'].mean().unstack(fill_value=0)

for segment in avg_prices.columns:
    fig.add_trace(
        go.Bar(
            name=f'{segment} Avg Price',
            x=avg_prices.index,
            y=avg_prices[segment],
            text=[f'Rs.{round(val/100000, 2)}L' for val in avg_prices[segment]],
            textposition='auto',
            showlegend=False
        ),
        row=1, col=2
    )

# 3. property types distribution
prop_type_city = df_filtered.groupby(['City', 'Type of Property']).size().unstack(fill_value=0)

for prop_type in prop_type_city.columns:
    fig.add_trace(
        go.Bar(
            name=prop_type,
            x=prop_type_city.index,
            y=prop_type_city[prop_type],
            showlegend=False
        ),
        row=2, col=1
    )

# 4. bedroom configuration
bedroom_city = df_filtered.groupby(['City', 'bedroom']).size().unstack(fill_value=0)

for bedroom in bedroom_city.columns:
    fig.add_trace(
        go.Bar(
            name=f'{bedroom} BHK',
            x=bedroom_city.index,
            y=bedroom_city[bedroom],
            showlegend=False
        ),
        row=2, col=2
    )

# 5. price distribution (Violin plots)
for city in df_filtered['City'].unique():
    city_data = df_filtered[df_filtered['City'] == city]
    fig.add_trace(
        go.Violin(
            y=city_data['Price'],
            name=city,
            box_visible=True,
            meanline_visible=True,
            showlegend=False
        ),
        row=3, col=1
    )

# 6. price per sqft comparison
price_per_sqft = df_filtered[df_filtered['Price_Per_Sqft'] > 0].groupby(['City', 'Property_Segment'])['Price_Per_Sqft'].mean().unstack(fill_value=0)

for segment in price_per_sqft.columns:
    fig.add_trace(
        go.Bar(
            name=f'{segment} Price/Sqft',
            x=price_per_sqft.index,
            y=price_per_sqft[segment],
            text=[f'Rs.{round(val)}' for val in price_per_sqft[segment]],
            textposition='auto',
            showlegend=False
        ),
        row=3, col=2
    )

# update layout
fig.update_layout(
    title_text="Mumbai vs Thane: Comprehensive Property Market Comparison",
    height=1200,
    showlegend=True,
    title_x=0.5,
    font=dict(size=11)
)

# update y-axes labels
fig.update_yaxes(title_text="Property Count", row=1, col=1)
fig.update_yaxes(title_text="Average Price (Rs.)", row=1, col=2)
fig.update_yaxes(title_text="Property Count", row=2, col=1)
fig.update_yaxes(title_text="Property Count", row=2, col=2)
fig.update_yaxes(title_text="Price (Rs.)", row=3, col=1)
fig.update_yaxes(title_text="Price per Sqft (Rs.)", row=3, col=2)


In [ ]:
# create subplot figure
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Residential: Price vs Carpet Area',
        'Commercial: Price vs Carpet Area',
        'Residential: Price Distribution by Bedroom',
        'Commercial: Property Types'
    ),
    specs=[[{"type": "scatter"}, {"type": "scatter"}],
            [{"type": "box"}, {"type": "bar"}]]
)

# 1. residential: Price vs Carpet Area
residential_data = df_filtered[(df_filtered['Property_Segment'] == 'Residential') & (df_filtered['Carpet Area'] > 0)]

for city in residential_data['City'].unique():
    city_res_data = residential_data[residential_data['City'] == city]
    fig.add_trace(
        go.Scatter(
            x=city_res_data['Carpet Area'],
            y=city_res_data['Price'],
            mode='markers',
            name=f'{city} Residential',
            marker=dict(
                size=8,
                opacity=0.6
            ),
            hovertemplate=f'<b>{city}</b><br>Area: %{{x}} sqft<br>Price: Rs.%{{round(y)}}<extra></extra>'
        ),
        row=1, col=1
    )

# 2. commercial: Price vs Carpet Area
commercial_data = df_filtered[(df_filtered['Property_Segment'] == 'Commercial') & (df_filtered['Carpet Area'] > 0)]

if len(commercial_data) > 0:
    for city in commercial_data['City'].unique():
        city_com_data = commercial_data[commercial_data['City'] == city]
        if len(city_com_data) > 0:
            fig.add_trace(
                go.Scatter(
                    x=city_com_data['Carpet Area'],
                    y=city_com_data['Price'],
                    mode='markers',
                    name=f'{city} Commercial',
                    marker=dict(
                        size=8,
                        opacity=0.6,
                        symbol='diamond'
                    ),
                    showlegend=False,
                    hovertemplate=f'<b>{city}</b><br>Area: %{{x}} sqft<br>Price: Rs.%{{round(y)}}<extra></extra>'
                ),
                row=1, col=2
            )

# 3. residential: Price distribution by bedroom
residential_bedroom = residential_data.groupby(['City', 'bedroom'])['Price'].apply(list).reset_index()

for city in residential_data['City'].unique():
    city_bedroom_data = residential_bedroom[residential_bedroom['City'] == city]
    for _, row in city_bedroom_data.iterrows():
        if len(row['Price']) > 0:
            fig.add_trace(
                go.Box(
                    y=row['Price'],
                    name=f"{city} {row['bedroom']}BHK",
                    boxpoints='outliers',
                    showlegend=False,
                ),
                row=2, col=1
            )

# 4. commercial: Property types
if len(commercial_data) > 0:
    commercial_types = commercial_data.groupby(['City', 'Type of Property']).size().unstack(fill_value=0)
    
    for prop_type in commercial_types.columns:
        fig.add_trace(
            go.Bar(
                name=prop_type,
                x=commercial_types.index,
                y=commercial_types[prop_type],
                showlegend=False
            ),
            row=2, col=2
        )

# update layout
fig.update_layout(
    title_text="Detailed Segment Analysis: Residential vs Commercial",
    height=800,
    showlegend=True,
    title_x=0.5
)

# update axes labels
fig.update_xaxes(title_text="Carpet Area (sqft)", row=1, col=1)
fig.update_xaxes(title_text="Carpet Area (sqft)", row=1, col=2)
fig.update_yaxes(title_text="Price (Rs.)", row=1, col=1)
fig.update_yaxes(title_text="Price (Rs.)", row=1, col=2)
fig.update_yaxes(title_text="Price (Rs.)", row=2, col=1)
fig.update_yaxes(title_text="Property Count", row=2, col=2)

# 3. Location-Based Premium Analysis [5 Marks]

In [20]:
# calculate price per sqft
df['Price_Per_Sqft'] = np.where(
    df['Carpet Area'] > 0,
    round(df['Price'] / df['Carpet Area'], 2),
    df.get('sqft Price ', 0)
)

# handle nulls in Area_Name
df['Area_Name'] = df['Area Name'].fillna('Unknown')

# standardize isPrimeLocationProperty column
df['Is_Prime_Location'] = df['isPrimeLocationProperty'].apply(
    lambda x: True if x in ['Y', 'Yes', '1', 1, True] else False
)

# print summary!
print(f"Data loaded successfully: {len(df)} properties")
print(f"Prime location properties: {df['Is_Prime_Location'].sum()}")
print(f"Non-prime location properties: {(~df['Is_Prime_Location']).sum()}")

Data loaded successfully: 12685 properties
Prime location properties: 9414
Non-prime location properties: 3271


In [ ]:
# filter high budget properties 
city_filtered_data = []

# print header for high-budget property summary
print(50 * '*')
print("* HIGH-BUDGET PROPERTY SUMMARY PER CITY *")
print(50 * '*')

# loop through each unique city in the dataset
for city in df['City'].unique():
    # extract data for current city
    city_data = df[df['City'] == city].copy()
    
    # process city data if it contains properties
    if len(city_data) > 0:
        # define high-budget threshold (top 70% of properties in the city)
        # note: Using 0.3 quantile means properties above 30th percentile (top 70%)
        budget_threshold = city_data['Price'].quantile(0.3)
        
        # filter properties above the budget threshold
        high_budget_data = city_data[city_data['Price'] >= budget_threshold].copy()

        # display summary statistics for current city
        print(f"{city} - High Budget Properties (>70th percentile):")
        print(f"  Threshold: Rs.{round(budget_threshold)}")
        print(f"  Total high-budget: {len(high_budget_data)}")
        print(f"  Prime locations: {high_budget_data['Is_Prime_Location'].sum()}")
        print(f"  Non-prime locations: {(~high_budget_data['Is_Prime_Location']).sum()}")
        print(50*"*")
        
        # add filtered data to collection
        city_filtered_data.append(high_budget_data)

# combine all high-budget properties from all cities into single dataframe
df_high_budget = pd.concat(city_filtered_data, ignore_index=True)

**************************************************
* HIGH-BUDGET PROPERTY SUMMARY PER CITY *
**************************************************
Thane - High Budget Properties (>70th percentile):
  Threshold: ₹6,000,000
  Total high-budget: 2863
  Prime locations: 1784
  Non-prime locations: 1079
**************************************************
Kalyan - High Budget Properties (>70th percentile):
  Threshold: ₹4,100,000
  Total high-budget: 6
  Prime locations: 3
  Non-prime locations: 3
**************************************************
Mumbai - High Budget Properties (>70th percentile):
  Threshold: ₹10,500,000
  Total high-budget: 5968
  Prime locations: 5060
  Non-prime locations: 908
**************************************************
Agartala - High Budget Properties (>70th percentile):
  Threshold: ₹5,117,000
  Total high-budget: 1
  Prime locations: 1
  Non-prime locations: 0
**************************************************
Palghar - High Budget Properties (>70th percentile):


In [ ]:
# create main comparison dashboard

# create subplot figure
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Price Comparison: Prime vs Non-Prime',
        'Price per Sqft Comparison',
        'Carpet Area Comparison',
        'Property Count by Location Type',
        'Price Distribution by Location',
        'Premium Analysis by City'
    ),
    specs=[[{"type": "bar"}, {"type": "bar"}],
            [{"type": "bar"}, {"type": "bar"}],
            [{"type": "violin"}, {"type": "bar"}]]
)

# assign colors per variabl for consistency
colors = {'Prime': 'red', 'Non-Prime': 'blue'}

# 1. average price comparison
price_comparison = df.groupby(['City', 'Is_Prime_Location'])['Price'].mean().unstack(fill_value=0)
price_comparison.columns = ['Non-Prime', 'Prime'] if False in price_comparison.columns else ['Prime']

for location_type in price_comparison.columns:
    is_prime = location_type == 'Prime'
    fig.add_trace(
        go.Bar(
            name=location_type,
            x=price_comparison.index,
            y=price_comparison[location_type],
            text=[f'Rs.{round(val/100000,2)}L' for val in price_comparison[location_type]],
            textposition='auto',
            marker_color=colors.get(location_type, '#333333')
        ),
        row=1, col=1
    )

# 2. price per sqft comparison
psf_data = df[df['Price_Per_Sqft'] > 0]
psf_comparison = psf_data.groupby(['City', 'Is_Prime_Location'])['Price_Per_Sqft'].mean().unstack(fill_value=0)
psf_comparison.columns = ['Non-Prime', 'Prime'] if False in psf_comparison.columns else ['Prime']

for location_type in psf_comparison.columns:
    fig.add_trace(
        go.Bar(
            name=location_type,
            x=psf_comparison.index,
            y=psf_comparison[location_type],
            text=[f'Rs.{round(val,2)}' for val in psf_comparison[location_type]],
            textposition='auto',
            marker_color=colors.get(location_type, '#333333'),
            showlegend=False
        ),
        row=1, col=2
    )

# 3. carpet area comparison
area_data = df[df['Carpet Area'] > 0]
area_comparison = area_data.groupby(['City', 'Is_Prime_Location'])['Carpet Area'].mean().unstack(fill_value=0)
area_comparison.columns = ['Non-Prime', 'Prime'] if False in area_comparison.columns else ['Prime']

for location_type in area_comparison.columns:
    fig.add_trace(
        go.Bar(
            name=location_type,
            x=area_comparison.index,
            y=area_comparison[location_type],
            text=[f'{round(val)} sqft' for val in area_comparison[location_type]],
            textposition='auto',
            marker_color=colors.get(location_type, '#333333'),
            showlegend=False
        ),
        row=2, col=1
    )

# 4. property count by location type
count_comparison = df.groupby(['City', 'Is_Prime_Location']).size().unstack(fill_value=0)
count_comparison.columns = ['Non-Prime', 'Prime'] if False in count_comparison.columns else ['Prime']

for location_type in count_comparison.columns:
    fig.add_trace(
        go.Bar(
            name=location_type,
            x=count_comparison.index,
            y=count_comparison[location_type],
            text=count_comparison[location_type],
            textposition='auto',
            marker_color=colors.get(location_type, '#333333'),
            showlegend=False
        ),
        row=2, col=2
    )

# 5. price distribution violin plots
for city in df['City'].unique():
    city_data = df[df['City'] == city]
    
    for is_prime in [True, False]:
        location_data = city_data[city_data['Is_Prime_Location'] == is_prime]
        location_type = 'Prime' if is_prime else 'Non-Prime'
        
        if len(location_data) > 0:
            fig.add_trace(
                go.Violin(
                    y=location_data['Price'],
                    name=f'{city} {location_type}',
                    box_visible=True,
                    meanline_visible=True,
                    showlegend=False,
                    line_color=colors.get(location_type, '#333333'),
                    opacity=0.7
                ),
                row=3, col=1
            )

# 6. premium percentage analysis
premium_analysis = []

for city in df['City'].unique():
    city_data = df[df['City'] == city]
    prime_avg = city_data[city_data['Is_Prime_Location'] == True]['Price'].mean()
    non_prime_avg = city_data[city_data['Is_Prime_Location'] == False]['Price'].mean()
    
    if not pd.isna(prime_avg) and not pd.isna(non_prime_avg) and non_prime_avg > 0:
        premium_percentage = ((prime_avg - non_prime_avg) / non_prime_avg) * 100
        premium_analysis.append({'City': city, 'Premium_Percentage': premium_percentage})

if premium_analysis:
    premium_df = pd.DataFrame(premium_analysis)
    fig.add_trace(
        go.Bar(
            x=premium_df['City'],
            y=premium_df['Premium_Percentage'],
            text=[f'{round(val,1)}%' for val in premium_df['Premium_Percentage']],
            textposition='auto',
            marker_color='#96CEB4',
            name='Location Premium',
            showlegend=False
        ),
        row=3, col=2
    )

# update layout
fig.update_layout(
    title_text="Location-Based Premium Analysis: Prime vs Non-Prime Properties",
    height=1200,
    showlegend=True,
    title_x=0.5,
    font=dict(size=11)
)

# update y-axes labels
fig.update_yaxes(title_text="Average Price (Rs.)", row=1, col=1)
fig.update_yaxes(title_text="Price per Sqft (Rs.)", row=1, col=2)
fig.update_yaxes(title_text="Average Area (sqft)", row=2, col=1)
fig.update_yaxes(title_text="Property Count", row=2, col=2)
fig.update_yaxes(title_text="Price (Rs.)", row=3, col=1)
fig.update_yaxes(title_text="Premium %", row=3, col=2)

In [23]:
# analyse amenities difference between prime and non-prime locations!!

amenity_analysis = {}

for city in df['City'].unique():
    city_data = df[df['City'] == city]
    
    prime_data = city_data[city_data['Is_Prime_Location'] == True]
    non_prime_data = city_data[city_data['Is_Prime_Location'] == False]
    
    city_amenity_stats = {}
    
    for location_type, data in [('Prime', prime_data), ('Non-Prime', non_prime_data)]:
        if len(data) == 0:
            continue
            
        amenity_stats = {}
        total_properties = len(data)
        
        for amenity in amenities_list:
            if amenity in data.columns:
                # Count amenities (handle different data types)
                if data[amenity].dtype == 'object':
                    positive_count = data[amenity].isin(['Y', 'Yes', '1', 1]).sum()
                else:
                    positive_count = (data[amenity] == 1).sum()
                
                percentage = (positive_count / total_properties) * 100 if total_properties > 0 else 0
                amenity_stats[amenity] = {
                    'count': positive_count,
                    'percentage': percentage
                }
        
        city_amenity_stats[location_type] = amenity_stats
    
    amenity_analysis[city] = city_amenity_stats

In [ ]:
# creating heatmap for amenities comparison

# Create separate heatmaps for each city
cities = list(amenity_analysis.keys())

fig = make_subplots(
    rows=len(cities), cols=1,
    subplot_titles=[f'{city} - Amenities Comparison' for city in cities],
    vertical_spacing=0.1
)

for i, city in enumerate(cities):
    city_data = amenity_analysis[city]
    
    # Prepare data for heatmap
    location_types = ['Prime', 'Non-Prime']
    z_data = []
    y_labels = []
    
    for location_type in location_types:
        if location_type in city_data:
            row_data = []
            for amenity in amenities_list:
                if amenity in city_data[location_type]:
                    row_data.append(city_data[location_type][amenity]['percentage'])
                else:
                    row_data.append(0)
            z_data.append(row_data)
            y_labels.append(location_type)
    
    if z_data:
        fig.add_trace(
            go.Heatmap(
                z=z_data,
                x=amenities_list,
                y=y_labels,
                colorscale='RdYlBu',
                text=[[f'{round(val,1)}%' for val in row] for row in z_data],
                texttemplate="%{text}",
                textfont={"size": 8},
                showscale=(i == 0),
                colorbar=dict(title="% of Properties") if i == 0 else None
            ),
            row=i+1, col=1
        )

fig.update_layout(
    title_text="Amenities Availability: Prime vs Non-Prime Locations",
    height=400 * len(cities),
    title_x=0.5
)

# 4. Value-for-Money Opportunities [10 Marks]


In [25]:
# filter properties with valid carpet area (essential for value analysis)
df = df[df['Carpet Area'] > 0].copy()

# calculate key value metrics
df['Sqft_Per_Lakh'] = df['Carpet Area'] / (df['Price'] / 100000)  # Area per lakh rupees
df['Value_Score'] = df['Sqft_Per_Lakh']  # Higher is better

print(f"Summary of the data to figure out value for money opportunities!!\n")
print(f"Total valid properties: {len(df)}")
print(f"Cities covered: {df['City'].nunique()}")
print(f"Price range: Rs.{round(df['Price'].min(),2)} - Rs.{round(df['Price'].max(),2)}")
print(f"Area range: {round(df['Carpet Area'].min(),2)} - {round(df['Carpet Area'].max(),2)} sqft")

Summary of the data to figure out value for money opportunities!!

Total valid properties: 10133
Cities covered: 7
Price range: Rs.100000.0 - Rs.40800000001.0
Area range: 2.0 - 20870.0 sqft


In [26]:
# identify top value-for-money properties across all cities!!

# change this number to get more properties
top_n = 100

# sort by value score (sqft per lakh) in descending order
top_properties = df.nlargest(top_n, 'Value_Score').copy()

# add ranking
top_properties['Overall_Rank'] = range(1, len(top_properties) + 1)

# calculate percentile ranking
df['Value_Percentile'] = df['Value_Score'].rank(pct=True) * 100

# display top 10 properties
display_cols = ['Overall_Rank', 'City', 'Area_Name', 'Price', 'Carpet Area', 
                'Price_Per_Sqft', 'Sqft_Per_Lakh', 'bedroom', 'Type of Property']

# print top 10 properties!!
print("***********************")
print("** TOP 10 PROPERTIES **")
print("***********************")
print(top_properties[display_cols].head(10).to_string(index=False))

***********************
** TOP 10 PROPERTIES **
***********************
 Overall_Rank   City       Area_Name      Price  Carpet Area  Price_Per_Sqft  Sqft_Per_Lakh  bedroom        Type of Property
            1 Mumbai      Dadar West   100000.0       6000.0           16.67    6000.000000        2               Apartment
            2 Mumbai  Kandivali East   125000.0        410.0          304.88     328.000000        1               Apartment
            3 Mumbai    Andheri West   410000.0       1340.0          305.97     326.829268        3               Apartment
            4 Mumbai Nalasopara East  1000000.0        650.0         1538.46      65.000000        2               Apartment
            5 Mumbai Nalasopara East  2000000.0        850.0         2352.94      42.500000        1 Builder Floor Apartment
            6 Mumbai      Vasai West   500000.0        200.0         2500.00      40.000000        1               Apartment
            7 Mumbai Nalasopara East   700000.0      

In [27]:
# let's create city-wise value rankings!

# initialise a dictionary to store data!
city_rankings = {}

for city in df['City'].unique():
    # get data for current city
    city_data = df[df['City'] == city].copy()
    
    # sort by value score and get top 20
    city_data_sorted = city_data.nlargest(20, 'Value_Score')
    city_data_sorted['City_Rank'] = range(1, len(city_data_sorted) + 1)
    
    # calculate city statistics
    city_stats = {
        'total_properties': len(city_data),
        'avg_value_score': city_data['Value_Score'].mean(),
        'median_value_score': city_data['Value_Score'].median(),
        'best_value_score': city_data['Value_Score'].max(),
        'avg_price_per_sqft': city_data['Price_Per_Sqft'].mean(),
        'median_price_per_sqft': city_data['Price_Per_Sqft'].median(),
        'avg_price': city_data['Price'].mean(),
        'top_properties': city_data_sorted
    }
    
    # store city stats in rankings
    city_rankings[city] = city_stats

In [ ]:
# create subplot figure!
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Top 20 Value Properties by City',
        'Value Score Distribution by City',
        'Price vs Carpet Area (Top Values)',
        'Value Score by Price Segment',
        'City-wise Average Value Scores',
        'Bedroom Configuration Value Analysis'
    ),
    specs=[[{"type": "bar"}, {"type": "violin"}],
            [{"type": "scatter"}, {"type": "box"}],
            [{"type": "bar"}, {"type": "bar"}]]
)

# set up consistent colors for each city
colors = px.colors.qualitative.Set3
city_colors = {city: colors[i % len(colors)] for i, city in enumerate(df['City'].unique())}

# first, let's see which cities have the most top value properties
top_20 = top_properties.head(20)
city_counts = top_20['City'].value_counts()

fig.add_trace(
    go.Bar(
        x=city_counts.index,
        y=city_counts.values,
        text=city_counts.values,
        textposition='auto',
        marker_color=[city_colors.get(city, '#333333') for city in city_counts.index],
        name='Top 20 Count'
    ),
    row=1, col=1
)

# now let's examine how value scores are distributed across different cities
for i, city in enumerate(df['City'].unique()):
    city_data = df[df['City'] == city]
    fig.add_trace(
        go.Violin(
            y=city_data['Value_Score'],
            name=city,
            box_visible=True,
            meanline_visible=True,
            line_color=city_colors.get(city, '#333333'),
            showlegend=False
        ),
        row=1, col=2
    )

# here's where we plot price vs area for the best value properties
# this helps identify if larger properties offer better value
top_100 = top_properties.head(100)
for city in top_100['City'].unique():
    city_top = top_100[top_100['City'] == city]
    fig.add_trace(
        go.Scatter(
            x=city_top['Carpet Area'],
            y=city_top['Price'],
            mode='markers',
            name=city,
            marker=dict(
                color=city_colors.get(city, '#333333'),
                size=8,
                opacity=0.7
            ),
            text=[f"Rank: {rank}<br>Value: {round(score,1)}" for rank, score in 
                    zip(city_top['Overall_Rank'], city_top['Value_Score'])],
            hovertemplate='<b>%{fullData.name}</b><br>Area: %{x} sqft<br>Price: Rs.%{Rs.(y)}<br>%{text}<extra></extra>',
            showlegend=False
        ),
        row=2, col=1
    )

# let's break down value scores by price segments to see patterns
# expensive doesn't always mean poor value
for city in df['City'].unique():
    city_data = df[df['City'] == city]
    for segment in city_data['Price_Segment'].unique():
        if pd.notna(segment):
            segment_data = city_data[city_data['Price_Segment'] == segment]
            fig.add_trace(
                go.Box(
                    y=segment_data['Value_Score'],
                    name=f'{city}-{segment}',
                    marker_color=city_colors.get(city, '#333333'),
                    showlegend=False,
                    opacity=0.7
                ),
                row=2, col=2
            )

# which city offers the best value on average? this will tell us
city_avg_values = df.groupby('City')['Value_Score'].mean().sort_values(ascending=False)

fig.add_trace(
    go.Bar(
        x=city_avg_values.index,
        y=city_avg_values.values,
        text=[f'{round(val,1)}' for val in city_avg_values.values],
        textposition='auto',
        marker_color=[city_colors.get(city, '#333333') for city in city_avg_values.index],
        name='Avg Value Score',
        showlegend=False
    ),
    row=3, col=1
)

# finally, let's see if bedroom count affects value proposition
# do studio apartments offer better value than 3bhk?
bedroom_value = df.groupby(['City', 'bedroom'])['Value_Score'].mean().unstack(fill_value=0)

for bedroom in bedroom_value.columns:
    fig.add_trace(
        go.Bar(
            name=f'{bedroom} BHK',
            x=bedroom_value.index,
            y=bedroom_value[bedroom],
            showlegend=True if bedroom <= 3 else False  # only show legend for common bedroom types
        ),
        row=3, col=2
    )

# make everything look nice and professional
fig.update_layout(
    title_text="Value-for-Money Analysis Dashboard",
    height=1200,
    showlegend=True,
    title_x=0.5,
    font=dict(size=11)
)

# add proper axis labels so everyone knows what they're looking at
fig.update_yaxes(title_text="Properties in Top 20", row=1, col=1)
fig.update_yaxes(title_text="Value Score (Sqft/Lakh)", row=1, col=2)
fig.update_xaxes(title_text="Carpet Area (sqft)", row=2, col=1)
fig.update_yaxes(title_text="Price (₹)", row=2, col=1)
fig.update_yaxes(title_text="Value Score", row=2, col=2)
fig.update_yaxes(title_text="Avg Value Score", row=3, col=1)
fig.update_yaxes(title_text="Avg Value Score", row=3, col=2)

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Value Score by Property Type',
        'Best Value Areas by City',
        'Price Segment Value Comparison',
        'Developer Value Analysis'
    ),
    specs=[[{"type": "box"}, {"type": "bar"}],
            [{"type": "bar"}, {"type": "bar"}]]
)

colors = px.colors.qualitative.Set2

# create value score distribution by property type using box plots
property_types = df['Type of Property'].value_counts().head(5).index

for i, prop_type in enumerate(property_types):
    type_data = df[df['Type of Property'] == prop_type]
    fig.add_trace(
        go.Box(
            y=type_data['Value_Score'],
            name=prop_type,
            marker_color=colors[i % len(colors)],
            boxpoints='outliers'
        ),
        row=1, col=1
    )

# find the best value areas by city (top 3 areas per city)
best_areas = []
for city in df['City'].unique():
    city_data = df[df['City'] == city]
    city_area_values = city_data.groupby('Area_Name')['Value_Score'].mean()
    top_areas = city_area_values.nlargest(3)
    
    # create formatted area names with city prefix for better identification
    for area, score in top_areas.items():
        best_areas.append({
            'City_Area': f"{city}-{area[:15]}",
            'City': city,
            'Area': area,
            'Value_Score': score
        })

# convert to dataframe and create bar chart for top areas
best_areas_df = pd.DataFrame(best_areas)
if len(best_areas_df) > 0:
    # show top 15 areas overall based on value score
    best_areas_df = best_areas_df.nlargest(15, 'Value_Score')
    
    fig.add_trace(
        go.Bar(
            x=best_areas_df['City_Area'],
            y=best_areas_df['Value_Score'],
            text=[f'{round(val)}' for val in best_areas_df['Value_Score']],
            textposition='auto',
            marker_color='#96CEB4',
            name='Area Value Score',
            showlegend=False
        ),
        row=1, col=2
    )

# compare value scores across different price segments by city
segment_values = df.groupby(['City', 'Price_Segment'])['Value_Score'].mean().unstack(fill_value=0)

# create grouped bar chart for price segment comparison
for segment in segment_values.columns:
    if pd.notna(segment):
        fig.add_trace(
            go.Bar(
                name=segment,
                x=segment_values.index,
                y=segment_values[segment]
            ),
            row=2, col=1
        )

# analyze developer performance if developer data is available
if 'Developer' in df.columns:
    developer_values = df.groupby('Developer')['Value_Score'].agg(['mean', 'count']).reset_index()
    # filter developers with at least 5 properties for reliable analysis
    developer_values = developer_values[developer_values['count'] >= 5]
    top_developers = developer_values.nlargest(10, 'mean')
    
    if len(top_developers) > 0:
        fig.add_trace(
            go.Bar(
                x=top_developers['Developer'],
                y=top_developers['mean'],
                text=[f'{round(val,1)}' for val in top_developers['mean']],
                textposition='auto',
                marker_color='#FECA57',
                name='Developer Value Score',
                showlegend=False
            ),
            row=2, col=2
        )

# update the overall layout and styling
fig.update_layout(
    title_text="Segment-wise Value Analysis",
    height=800,
    showlegend=True,
    title_x=0.5
)

# set y-axis labels for all subplots
fig.update_yaxes(title_text="Value Score", row=1, col=1)
fig.update_yaxes(title_text="Value Score", row=1, col=2)
fig.update_yaxes(title_text="Value Score", row=2, col=1)
fig.update_yaxes(title_text="Value Score", row=2, col=2)

# rotate x-axis labels for better readability on crowded charts
fig.update_xaxes(tickangle=45, row=1, col=2)
fig.update_xaxes(tickangle=45, row=2, col=2)

In [ ]:
# select top 25 properties from our ranked list
top_25 = top_properties.head(25)

# create a detailed table to showcase the best investment opportunities
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['Rank', 'City', 'Area', 'Price (₹)', 'Area (sqft)', 
                'Price/Sqft', 'Value Score', 'BHK', 'Type'],
        fill_color='darkblue',
        font=dict(color='white', size=12),
        align='center',
        height=40
    ),
    cells=dict(
        values=[
            top_25['Overall_Rank'],
            top_25['City'],
            [area[:20] + '...' if len(area) > 20 else area for area in top_25['Area_Name']],
            ['Rs.' + f"{round(price)}" for price in top_25['Price']],
            [f"{round(area)}" for area in top_25['Carpet Area']],
            ['Rs.' + f"{round(psf)}" for psf in top_25['Price_Per_Sqft']],
            [f"{round(score,1)}" for score in top_25['Value_Score']],
            top_25['bedroom'],
            [ptype[:15] for ptype in top_25['Type of Property']]
        ],
        # alternate row colors for better readability
        fill_color=[['lightgray' if i % 2 == 0 else 'white' for i in range(len(top_25))]],
        align='center',
        font=dict(size=10),
        height=30
    )
)])

# customize the table layout and appearance
fig.update_layout(
    title="Top 25 Value-for-Money Investment Opportunities",
    height=800,
    title_x=0.5
)

## 5. Analyse how various amenities (e.g. Swimming Pool, Gymnasium, Club House) affect property prices. Make a comparative analysis of these features in the cities of Mumbai and Thane. Determine which features add the most value and whether preferences differ across cities. [5 Marks]

In [31]:
# filter for mumbai and thane only
df_filtered = df[df['City'].isin(['Mumbai', 'Thane'])].copy()

# filter available amenities and standardize values
available_amenities = []
for col in amenities_list:
    if col in df_filtered.columns:
        # standardize amenity values to binary (1 for yes, 0 for no)
        df_filtered[col] = df_filtered[col].apply(
            lambda x: 1 if x in ['Y', 'Yes', '1', 1, True] else 0
        )
        available_amenities.append(col)

# calculate amenity scores
df_filtered['Total_Amenities'] = df_filtered[available_amenities].sum(axis=1)
df_filtered['Amenity_Percentage'] = df_filtered['Total_Amenities'] / len(available_amenities) * 100

print(f"Summary:")
print(f"Mumbai properties: {len(df_filtered[df_filtered['City'] == 'Mumbai'])}")
print(f"Thane properties: {len(df_filtered[df_filtered['City'] == 'Thane'])}")
print(f"Total properties for analysis: {len(df_filtered)}")
print(f"Available amenities: {len(available_amenities)}")

Summary:
Mumbai properties: 7123
Thane properties: 2997
Total properties for analysis: 10120
Available amenities: 90


In [32]:
# calculate simple price differences for properties with vs without each amenity

amenity_results = {}

# loop through mumbai and thane cities for comparison
for city in ['Mumbai', 'Thane']:
    # get data for current city only
    city_data = df_filtered[df_filtered['City'] == city]
    city_results = {}
    
    # analyze each available amenity
    for amenity in available_amenities:
        # split properties based on whether they have the amenity or not
        with_amenity = city_data[city_data[amenity] == 1]
        without_amenity = city_data[city_data[amenity] == 0]
        
        # only proceed if we have enough data points in both groups
        if len(with_amenity) >= 3 and len(without_amenity) >= 3:
            # calculate average prices for both groups
            avg_price_with = with_amenity['Price'].mean()
            avg_price_without = without_amenity['Price'].mean()
            avg_psf_with = with_amenity['Price_Per_Sqft'].mean()
            avg_psf_without = without_amenity['Price_Per_Sqft'].mean()
            
            # calculate how much more expensive properties with amenities are
            price_diff_percent = ((avg_price_with - avg_price_without) / avg_price_without * 100) if avg_price_without > 0 else 0
            psf_diff_percent = ((avg_psf_with - avg_psf_without) / avg_psf_without * 100) if avg_psf_without > 0 else 0
            
            # calculate what percentage of properties in this city have this amenity
            adoption_rate = (len(with_amenity) / len(city_data)) * 100
            
            # store all the calculated metrics for this amenity
            city_results[amenity] = {
                'avg_price_with': avg_price_with,
                'avg_price_without': avg_price_without,
                'price_difference_percent': price_diff_percent,
                'avg_psf_with': avg_psf_with,
                'avg_psf_without': avg_psf_without,
                'psf_difference_percent': psf_diff_percent,
                'adoption_rate': adoption_rate,
                'properties_with': len(with_amenity),
                'properties_without': len(without_amenity)
            }
    
    # store results for this city
    amenity_results[city] = city_results

In [33]:
"""Create visualizations showing relationship between price and amenities"""

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Price Distribution by Amenity Count',
        'Price per Sqft vs Amenity Percentage',
        'City-wise Amenity vs Price Analysis',
        'Premium Properties Amenity Profile'
    ),
    specs=[[{"type": "violin"}, {"type": "scatter"}],
            [{"type": "bar"}, {"type": "bar"}]]
)

colors = {'Mumbai': '#FF6B6B', 'Thane': '#4ECDC4'}

# 1. Price distribution by amenity count (violin plots)
# Group amenities into categories
df_filtered['Amenity_Range'] = pd.cut(df_filtered['Total_Amenities'], 
                            bins=[0, 3, 6, 9, 20], 
                            labels=['0-3 Amenities', '4-6 Amenities', '7-9 Amenities', '10+ Amenities'])

for amenity_range in df_filtered['Amenity_Range'].unique():
    if pd.notna(amenity_range):
        range_data = df_filtered[df_filtered['Amenity_Range'] == amenity_range]
        
        fig.add_trace(
            go.Violin(
                y=range_data['Price'],
                name=str(amenity_range),
                box_visible=True,
                meanline_visible=True
            ),
            row=1, col=1
        )

# 2. Price per sqft vs amenity percentage
for city in ['Mumbai', 'Thane']:
    city_data = df_filtered[df_filtered['City'] == city]
    
    fig.add_trace(
        go.Scatter(
            x=city_data['Amenity_Percentage'],
            y=city_data['Price_Per_Sqft'],
            mode='markers',
            name=city,
            marker=dict(
                color=colors[city],
                size=6,
                opacity=0.6
            ),
            showlegend=False
        ),
        row=1, col=2
    )

# 3. City-wise amenity vs price analysis
amenity_bins = ['0-25%', '26-50%', '51-75%', '76-100%']
df_filtered['Amenity_Bin'] = pd.cut(df_filtered['Amenity_Percentage'], 
                            bins=[0, 25, 50, 75, 100], 
                            labels=amenity_bins)

city_amenity_price = df_filtered.groupby(['City', 'Amenity_Bin'])['Price'].mean().unstack(fill_value=0)

for amenity_bin in amenity_bins:
    if amenity_bin in city_amenity_price.columns:
        fig.add_trace(
            go.Bar(
                name=amenity_bin,
                x=city_amenity_price.index,
                y=city_amenity_price[amenity_bin] / 100000  # Convert to lakhs
            ),
            row=2, col=1
        )

# 4. Premium properties (top 25% by price) amenity profile
price_75th = df_filtered['Price'].quantile(0.75)
premium_properties = df_filtered[df_filtered['Price'] >= price_75th]

if len(premium_properties) > 0:
    premium_amenity_avg = {}
    regular_amenity_avg = {}
    regular_properties = df_filtered[df_filtered['Price'] < price_75th]
    
    # Calculate average amenity percentage for premium vs regular
    for city in ['Mumbai', 'Thane']:
        premium_city = premium_properties[premium_properties['City'] == city]
        regular_city = regular_properties[regular_properties['City'] == city]
        
        premium_amenity_avg[city] = premium_city['Amenity_Percentage'].mean() if len(premium_city) > 0 else 0
        regular_amenity_avg[city] = regular_city['Amenity_Percentage'].mean() if len(regular_city) > 0 else 0
    
    cities = list(premium_amenity_avg.keys())
    premium_values = list(premium_amenity_avg.values())
    regular_values = list(regular_amenity_avg.values())
    
    fig.add_trace(
        go.Bar(
            name='Premium Properties',
            x=cities,
            y=premium_values,
            marker_color='gold',
            opacity=0.8
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Bar(
            name='Regular Properties',
            x=cities,
            y=regular_values,
            marker_color='silver',
            opacity=0.8
        ),
        row=2, col=2
    )

# Update layout
fig.update_layout(
    title_text="Price vs Amenities Relationship Analysis",
    height=1000,
    showlegend=True,
    title_x=0.5
)

# Update axes labels
fig.update_yaxes(title_text="Price (₹)", row=1, col=1)
fig.update_xaxes(title_text="Amenity Percentage", row=1, col=2)
fig.update_yaxes(title_text="Price per Sqft (₹)", row=1, col=2)
fig.update_yaxes(title_text="Avg Price (₹ Lakhs)", row=2, col=1)
fig.update_yaxes(title_text="Avg Amenity %", row=2, col=2)

# 6. Timeline & Readiness Effect on Pricing [10 Marks]

In [34]:
df_filtered.columns

Index(['ID', 'Possession Status', 'Availability Starts From', 'Floor No',
       'Commercial', 'Developer', 'Approved Authority Name', 'Units Available',
       'Price', 'Price (English)',
       ...
       'Price_Per_Sqft', 'Area_Name', 'Is_Prime_Location', 'Sqft_Per_Lakh',
       'Value_Score', 'Value_Percentile', 'Total_Amenities',
       'Amenity_Percentage', 'Amenity_Range', 'Amenity_Bin'],
      dtype='object', length=156)

In [35]:
df_filtered['Possession Status'].unique()

array(['Under Construction', 'Ready to Move', "Jun '26", "Dec '28",
       "Nov '25", "Dec '26", "Mar '25", "Dec '23", "Jun '24", "Dec '24",
       "Sep '23", "Mar '24", nan, "Dec '27", "Dec '25", "Apr '28",
       "Nov '23", "Apr '24", "Jun '25", "Sep '26"], dtype=object)

In [36]:
def parse_availability_dates(df):
    """Parse availability dates and calculate months to availability"""
    
    current_date = datetime.now()
    
    def parse_availability(availability_str):
        availability_str = str(availability_str).strip().lower()
        
        # check if they're ready to start immediately
        if any(word in availability_str for word in ['immediate', 'ready', 'na', 'nan', 'available']):
            return 0
        
        # look for december mentions with specific years
        if 'dec' in availability_str or 'december' in availability_str:
            if '25' in availability_str or '2025' in availability_str:
                target_date = datetime(2025, 12, 1)
            elif '24' in availability_str or '2024' in availability_str:
                target_date = datetime(2024, 12, 1)
            elif '26' in availability_str or '2026' in availability_str:
                target_date = datetime(2026, 12, 1)
            else:
                target_date = datetime(2025, 12, 1)  # assume 2025 if year isn't clear
            
            # calculate how many months from now
            months_diff = (target_date.year - current_date.year) * 12 + (target_date.month - current_date.month)
            return max(0, months_diff)
        
        # same logic for march dates
        if 'mar' in availability_str or 'march' in availability_str:
            if '25' in availability_str or '2025' in availability_str:
                target_date = datetime(2025, 3, 1)
            elif '24' in availability_str or '2024' in availability_str:
                target_date = datetime(2024, 3, 1)
            elif '26' in availability_str or '2026' in availability_str:
                target_date = datetime(2026, 3, 1)
            else:
                target_date = datetime(2025, 3, 1)
            
            months_diff = (target_date.year - current_date.year) * 12 + (target_date.month - current_date.month)
            return max(0, months_diff)
        
        # handle january mentions
        if 'jan' in availability_str or 'january' in availability_str:
            if '25' in availability_str or '2025' in availability_str:
                target_date = datetime(2025, 1, 1)
            elif '26' in availability_str or '2026' in availability_str:
                target_date = datetime(2026, 1, 1)
            else:
                target_date = datetime(2025, 1, 1)
            
            months_diff = (target_date.year - current_date.year) * 12 + (target_date.month - current_date.month)
            return max(0, months_diff)
        
        # when only year is mentioned, make reasonable guesses
        if '2024' in availability_str:
            return max(0, 3)  # probably mid-2024, so just a few months left
        elif '2025' in availability_str:
            return 6  # middle of 2025 seems reasonable
        elif '2026' in availability_str:
            return 18  # middle of 2026
        elif '2027' in availability_str:
            return 30  # middle of 2027
        
        # fallback for when we can't figure it out
        return 12  # just assume they need about a year
    
    df['Months_to_Availability'] = df['Availability_Raw'].apply(parse_availability)
    return df

In [37]:
def calculate_readiness_score(row):
    """Calculate readiness score based on possession status and timeline"""
    
    months = row['Months_to_Availability']
    possession = row['Possession_Category']
    
    # first, let's score based on how soon the property will be available
    if months == 0:
        timeline_score = 5  # available immediately - highest score
    elif months <= 6:
        timeline_score = 4  # within 6 months - pretty good
    elif months <= 12:
        timeline_score = 3  # within a year - decent
    elif months <= 24:
        timeline_score = 2  # within 2 years - not ideal but ok
    else:
        timeline_score = 1  # more than 2 years - lowest score
    
    # now adjust the score based on possession status
    if possession == 'Ready to Move':
        return min(5, timeline_score + 1)  # boost score for ready properties, cap at 5
    elif possession == 'Under Construction':
        return max(1, timeline_score - 1)  # reduce score for construction risk, floor at 1
    else:
        return timeline_score  # keep original score for other categories

In [38]:
def categorize_timeline(months):
    """categorize timeline into meaningful buckets"""
    
    # handle the special case where there's no waiting time at all
    if months == 0:
        return 'Immediate (0 months)'
    # for quick turnarounds within half a year
    elif months <= 6:
        return 'Short-term (1-6 months)'
    # for planning that spans most of a year
    elif months <= 12:
        return 'Medium-term (7-12 months)'
    # for commitments that extend beyond a year but stay reasonable
    elif months <= 24:
        return 'Long-term (1-2 years)'
    # for really long commitments that require serious patience
    else:
        return 'Very Long-term (2+ years)'

In [39]:
# df_filtered already has data only from thane and mumbai
# clean possession status
df_filtered['Possession_Status'] = df_filtered['Possession Status'].fillna('Unknown')

# standardize possession status values
def standardize_possession(status):
    status_str = str(status).lower()
    if any(word in status_str for word in ['ready', 'move']):
        return 'Ready to Move'
    elif any(word in status_str for word in ['under', 'construction']):
        return 'Under Construction'
    else:
        return 'Other'

df_filtered['Possession_Category'] = df_filtered['Possession_Status'].apply(standardize_possession)

# process availability dates
df_filtered['Availability_Raw'] = df_filtered['Availability Starts From'].fillna('Immediate')

# parse and categorize availability dates
df_filtered = parse_availability_dates(df_filtered)

# create readiness score (immediate = 5, 6 months = 4, 1 year = 3, etc.)
df_filtered['Readiness_Score'] = df_filtered.apply(calculate_readiness_score, axis=1)

# create timeline categories for analysis
df_filtered['Timeline_Category'] = df_filtered['Months_to_Availability'].apply(categorize_timeline)

print(f"Summary:")
print(f"Mumbai properties: {len(df_filtered[df_filtered['City'] == 'Mumbai'])}")
print(f"Thane properties: {len(df_filtered[df_filtered['City'] == 'Thane'])}")
print(f"Possession categories: {df_filtered['Possession_Category'].value_counts().to_dict()}")


Summary:
Mumbai properties: 7123
Thane properties: 2997
Possession categories: {'Ready to Move': 7328, 'Under Construction': 2541, 'Other': 251}


In [40]:
# create comprehensive possession status analysis dashboard

# create subplot layout with 2x2 grid for possession status analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Price Comparison by Possession Status',
        'Price Distribution by Possession Status',
        'Price per Sqft by Possession Status',
        'Property Count by Possession Status'
    ),
    specs=[[{"type": "bar"}, {"type": "violin"}],
            [{"type": "bar"}, {"type": "bar"}]]
)

# define color palette for cities and possession categories
colors = {'Mumbai': '#FF6B6B', 'Thane': '#4ECDC4', 
            'Ready to Move': '#2ECC71', 'Under Construction': '#E74C3C', 
            'New Launch': '#F39C12', 'Other': '#95A5A6'}

# 1. calculate average price by city and possession status
possession_price = df_filtered.groupby(['City', 'Possession_Category'])['Price'].mean().unstack(fill_value=0)

# add bar traces for price comparison across possession categories
for possession_status in possession_price.columns:
    fig.add_trace(
        go.Bar(
            name=possession_status,
            x=possession_price.index,
            y=possession_price[possession_status] / 100000,  # convert to lakhs for readability
            text=[f'Rs.{round(val/100000,2)}L' for val in possession_price[possession_status]],
            textposition='auto',
            marker_color=colors.get(possession_status, '#333333'),
            opacity=0.8
        ),
        row=1, col=1
    )

# 2. create violin plots to show price distribution by possession status
for possession_status in df_filtered['Possession_Category'].unique():
    # only include categories with sufficient data points
    if possession_status != 'Other' or df_filtered[df_filtered['Possession_Category'] == possession_status].shape[0] > 10:
        status_data = df_filtered[df_filtered['Possession_Category'] == possession_status]
        
        fig.add_trace(
            go.Violin(
                y=status_data['Price'],
                name=possession_status,
                box_visible=True,
                meanline_visible=True,
                line_color=colors.get(possession_status, '#333333'),
                showlegend=False
            ),
            row=1, col=2
        )

# 3. analyze price per square foot by possession status
possession_psf = df_filtered.groupby(['City', 'Possession_Category'])['Price_Per_Sqft'].mean().unstack(fill_value=0)

# add bar traces for price per sqft comparison
for possession_status in possession_psf.columns:
    fig.add_trace(
        go.Bar(
            name=possession_status,
            x=possession_psf.index,
            y=possession_psf[possession_status],
            text=[f'Rs.{round(val,2)}' for val in possession_psf[possession_status]],
            textposition='auto',
            marker_color=colors.get(possession_status, '#333333'),
            opacity=0.8,
            showlegend=False
        ),
        row=2, col=1
    )

# 4. count properties by possession status to understand market composition
possession_count = df_filtered.groupby(['City', 'Possession_Category']).size().unstack(fill_value=0)

# add bar traces for property count comparison
for possession_status in possession_count.columns:
    fig.add_trace(
        go.Bar(
            name=possession_status,
            x=possession_count.index,
            y=possession_count[possession_status],
            text=possession_count[possession_status],
            textposition='auto',
            marker_color=colors.get(possession_status, '#333333'),
            opacity=0.8,
            showlegend=False
        ),
        row=2, col=2
    )

# customize overall layout and appearance
fig.update_layout(
    title_text="Possession Status Impact Analysis: Mumbai vs Thane",
    height=1000,
    showlegend=True,
    title_x=0.5
)

# add descriptive labels to y-axes for better understanding
fig.update_yaxes(title_text="Average Price (Rs. Lakhs)", row=1, col=1)
fig.update_yaxes(title_text="Price (Rs.)", row=1, col=2)
fig.update_yaxes(title_text="Price per Sqft (Rs.)", row=2, col=1)
fig.update_yaxes(title_text="Property Count", row=2, col=2)

In [ ]:
"""Create timeline-based analysis dashboard"""

# create a subplot
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Price by Timeline Category',
        'Months to Availability vs Price',
        'Readiness Score Impact',
        'Timeline Distribution'
    ),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
            [{"type": "bar"}, {"type": "bar"}]]
)

# define colors for each city to maintain consistency across plots
colors = {'Mumbai': '#FF6B6B', 'Thane': '#4ECDC4'}

# 1. analyze average price by timeline category for each city
timeline_price = df_filtered.groupby(['City', 'Timeline_Category'])['Price'].mean().unstack(fill_value=0)

# arrange timeline categories in logical order from immediate to long-term
timeline_order = ['Immediate (0 months)', 'Short-term (1-6 months)', 
                    'Medium-term (7-12 months)', 'Long-term (1-2 years)', 
                    'Very Long-term (2+ years)']

# reorder columns based on timeline progression
timeline_price = timeline_price.reindex(columns=[col for col in timeline_order if col in timeline_price.columns])

# create bar chart for each timeline category showing prices by city
for timeline_cat in timeline_price.columns:
    fig.add_trace(
        go.Bar(
            name=timeline_cat,
            x=timeline_price.index,
            y=timeline_price[timeline_cat] / 100000,  # convert to lakhs for readability
            text=[f'Rs.{round(val/100000,2)}L' for val in timeline_price[timeline_cat]]
        ),
        row=1, col=1
    )

# 2. create scatter plot to show relationship between availability timeline and price
for city in ['Mumbai', 'Thane']:
    city_data = df_filtered[df_filtered['City'] == city]
    
    fig.add_trace(
        go.Scatter(
            x=city_data['Months_to_Availability'],
            y=city_data['Price'],
            mode='markers',
            name=city,
            marker=dict(
                color=colors[city],
                size=6,
                opacity=0.6
            ),
            hovertemplate=f'<b>{city}</b><br>Months: %{{x}}<br>Price: Rs.%{{round(y)}}<extra></extra>',
            showlegend=False
        ),
        row=1, col=2
    )

# 3. analyze how readiness score affects pricing in each city
readiness_price = df_filtered.groupby(['City', 'Readiness_Score'])['Price'].mean().unstack(fill_value=0)

# create bars for each city showing price variation by readiness score
for city in ['Mumbai', 'Thane']:
    if city in readiness_price.index:
        fig.add_trace(
            go.Bar(
                name=city,
                x=list(readiness_price.columns),
                y=readiness_price.loc[city] / 100000,  # convert to lakhs
                marker_color=colors[city],
                opacity=0.8,
                showlegend=False
            ),
            row=2, col=1
        )

# 4. show distribution of properties across timeline categories by city
timeline_dist = df_filtered.groupby(['City', 'Timeline_Category']).size().unstack(fill_value=0)
timeline_dist = timeline_dist.reindex(columns=[col for col in timeline_order if col in timeline_dist.columns])

# create bars showing count of properties in each timeline category
for city in ['Mumbai', 'Thane']:
    if city in timeline_dist.index:
        fig.add_trace(
            go.Bar(
                name=city,
                x=list(timeline_dist.columns),
                y=timeline_dist.loc[city],
                marker_color=colors[city],
                opacity=0.8,
                text=timeline_dist.loc[city],
                textposition='auto',
                showlegend=False
            ),
            row=2, col=2
        )

# customize overall layout and appearance
fig.update_layout(
    title_text="Timeline Impact Analysis: Availability Dates & Readiness",
    height=1000,
    showlegend=True,
    title_x=0.5
)

# set appropriate axis labels for each subplot
fig.update_yaxes(title_text="Average Price (Rs. Lakhs)", row=1, col=1)
fig.update_xaxes(title_text="Months to Availability", row=1, col=2)
fig.update_yaxes(title_text="Price (Rs.)", row=1, col=2)
fig.update_xaxes(title_text="Readiness Score", row=2, col=1)
fig.update_yaxes(title_text="Average Price (Rs. Lakhs)", row=2, col=1)
fig.update_yaxes(title_text="Property Count", row=2, col=2)

# rotate x-axis labels to prevent overlap and improve readability
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=2, col=2)

# 7. Developer Impact on Properties [15 Marks]

In [ ]:
# prepare the dataset for developer analysis

# clean developer names and create a new column
df['Developer_Clean'] = df['Developer'].fillna('Unknown Developer')

# standardize developer names by removing extra spaces and fixing common variations
df['Developer_Clean'] = df['Developer_Clean'].str.strip()
df['Developer_Clean'] = df['Developer_Clean'].replace('NA', 'Unknown Developer')
df['Developer_Clean'] = df['Developer_Clean'].replace('', 'Unknown Developer')

# convert amenity columns to binary format for easier analysis
for col in amenities_list:
    if col in df.columns:
        # convert to binary: 1 for yes/y/1/true, 0 for everything else
        df[col] = df[col].apply(
            lambda x: 1 if str(x).upper() in ['Y', 'YES', '1', 'TRUE'] else 0
        )
        available_amenities.append(col)

# calculate total amenities per property and create a percentage score
df['Total_Amenities'] = df[available_amenities].sum(axis=1)
df['Amenity_Score'] = (df['Total_Amenities'] / len(available_amenities)) * 100

# filter developers with minimum properties for meaningful analysis
developer_counts = df['Developer_Clean'].value_counts()
significant_developers = developer_counts[developer_counts >= 5].index.tolist()

# keep top developers and group the rest together
top_developers = developer_counts.head(15).index.tolist()

df['Developer_Category'] = df['Developer_Clean'].apply(
    lambda x: x if x in top_developers else 'Other Developers'
)

# print summary
print(f"Summary:")
print(f"Total properties: {len(df)}")
print(f"Total developers: {df['Developer_Clean'].nunique()}")
print(f"Top developers (≥5 properties): {len(significant_developers)}")
print(f"Available amenities: {len(available_amenities)}")

Summary:
Total properties: 10133
Total developers: 801
Top developers (≥5 properties): 224
Available amenities: 180


In [ ]:
# create comprehensive developer overview dashboard

# create the subplot structure with 4 different visualization types
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Top Developers by Average Price',
        'Developer Portfolio Size vs Average Price',
        'Price Distribution by Top Developers',
        'Developer Market Share'
    ),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
            [{"type": "violin"}, {"type": "pie"}]]
)

# first, let's calculate key metrics for each significant developer
developer_stats = df[df['Developer_Clean'].isin(significant_developers)].groupby('Developer_Clean').agg({
    'Price': ['mean', 'count'],
    'Price_Per_Sqft': 'mean'
}).round(0)

# clean up the column names for easier access
developer_stats.columns = ['Avg_Price', 'Property_Count', 'Avg_PSF']
top_price_developers = developer_stats.nlargest(12, 'Avg_Price')

# 1. bar chart showing which developers command the highest average prices
fig.add_trace(
    go.Bar(
        x=top_price_developers.index,
        y=top_price_developers['Avg_Price'] / 100000,  # convert to lakhs for readability
        text=[f'Rs.{round(val/100000,2)}L' for val in top_price_developers['Avg_Price']],
        textposition='auto',
        marker_color='darkblue',
        opacity=0.8,
        name='Avg Price'
    ),
    row=1, col=1
)

# 2. scatter plot to see if there's a relationship between portfolio size and pricing power
for developer in significant_developers[:10]:  # limit to top 10 for visual clarity
    dev_data = developer_stats.loc[developer]
    
    fig.add_trace(
        go.Scatter(
            x=[dev_data['Property_Count']],
            y=[dev_data['Avg_Price']],
            mode='markers+text',
            text=[developer[:15] + '...' if len(developer) > 15 else developer],
            textposition='top center',
            marker=dict(
                size=10,
                opacity=0.7
            ),
            name=developer,
            showlegend=False,
            hovertemplate=f'<b>{developer}</b><br>Properties: %{{x}}<br>Avg Price: ₹%{{round(y)}}<extra></extra>'
        ),
        row=1, col=2
    )

# 3. violin plots to show price distributions for the most active developers
top_volume_developers = developer_stats.nlargest(6, 'Property_Count')

for developer in top_volume_developers.index:
    dev_properties = df[df['Developer_Clean'] == developer]
    
    fig.add_trace(
        go.Violin(
            y=dev_properties['Price'],
            name=developer[:20] + '...' if len(developer) > 20 else developer,
            box_visible=True,
            meanline_visible=True,
            showlegend=False
        ),
        row=2, col=1
    )

# 4. pie chart showing market share distribution across developer categories
market_share = df['Developer_Category'].value_counts().head(10)

fig.add_trace(
    go.Pie(
        labels=[label[:15] + '...' if len(label) > 15 else label for label in market_share.index],
        values=market_share.values,
        name="Market Share",
        hole=0.3
    ),
    row=2, col=2
)

# polish the overall layout
fig.update_layout(
    title_text="Developer Impact Overview Analysis",
    height=1200,
    showlegend=True,
    title_x=0.5
)

# add proper axis labels for better understanding
fig.update_yaxes(title_text="Average Price (Rs. Lakhs)", row=1, col=1)
fig.update_xaxes(title_text="Number of Properties", row=1, col=2)
fig.update_yaxes(title_text="Price (Rs.)", row=1, col=2)
fig.update_yaxes(title_text="Price (Rs.)", row=2, col=1)

# make the developer names readable by rotating them
fig.update_xaxes(tickangle=45, row=1, col=1)

In [ ]:
# create analysis of developer premium positioning

# create a 2x2 subplot layout for comprehensive developer analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Developer Price Segments Distribution',
        'Premium Developers (>₹1Cr Average)',
        'Price per Sqft by Developer Category',
        'Developer Quality Score Matrix'
    ),
    specs=[[{"type": "bar"}, {"type": "bar"}],
            [{"type": "bar"}, {"type": "scatter"}]]
)

# define color scheme for different price segments
colors = {'Budget (<50L)': '#3498DB', 'Mid-Range (50L-1Cr)': '#F39C12', 'Premium (>1Cr)': '#E74C3C'}

# 1. analyze price segment distribution for top developers
# get the most active developers by property count
top_developers = df['Developer_Clean'].value_counts().head(8).index
# create cross-tabulation of developers vs price segments
segment_distribution = df[df['Developer_Clean'].isin(top_developers)].groupby(
    ['Developer_Clean', 'Price_Segment']).size().unstack(fill_value=0)

# add stacked bars for each price segment
for segment in segment_distribution.columns:
    if pd.notna(segment):
        fig.add_trace(
            go.Bar(
                name=segment,
                x=segment_distribution.index,
                y=segment_distribution[segment],
                marker_color=colors.get(segment, '#333333'),
                opacity=0.8
            ),
            row=1, col=1
        )

# 2. identify and visualize premium developers (those with >1cr average pricing)
premium_threshold = 10000000  # 1 crore in rupees
# calculate average price and property count for significant developers
developer_stats = df[df['Developer_Clean'].isin(significant_developers)].groupby('Developer_Clean').agg({
    'Price': ['mean', 'count']
})
developer_stats.columns = ['Avg_Price', 'Count']

# filter for premium developers and get top 10
premium_developers = developer_stats[developer_stats['Avg_Price'] > premium_threshold]
premium_developers = premium_developers.sort_values('Avg_Price', ascending=False).head(10)

# plot premium developers if any exist
if len(premium_developers) > 0:
    fig.add_trace(
        go.Bar(
            x=premium_developers.index,
            y=premium_developers['Avg_Price'] / 10000000,  # convert to crores for readability
            text=[f'Rs.{round(val/10000000,2)}Cr' for val in premium_developers['Avg_Price']],
            textposition='auto',
            marker_color='darkred',
            opacity=0.8,
            name='Premium Developers'
        ),
        row=1, col=2
    )

# 3. analyze price per sqft by developer pricing category
# calculate average price for each developer
developer_avg_price = df.groupby('Developer_Clean')['Price'].mean()

# function to categorize developers based on their average pricing
def categorize_developer(developer):
    if developer not in developer_avg_price.index:
        return 'Unknown'
    avg_price = developer_avg_price[developer]
    if avg_price > 15000000:  # >1.5cr
        return 'Ultra Premium'
    elif avg_price > 10000000:  # >1cr
        return 'Premium'
    elif avg_price > 5000000:   # >50l
        return 'Mid-Range'
    else:
        return 'Budget'

# apply categorization to all properties
df['Developer_Price_Category'] = df['Developer_Clean'].apply(categorize_developer)

# calculate average price per sqft by category
category_psf = df.groupby('Developer_Price_Category')['Price_Per_Sqft'].mean().sort_values(ascending=False)

# plot price per sqft by category
fig.add_trace(
    go.Bar(
        x=category_psf.index,
        y=category_psf.values,
        text=[f'Rs.{round(val)}' for val in category_psf.values],
        textposition='auto',
        marker_color='darkorange',
        opacity=0.8,
        name='Avg PSF'
    ),
    row=2, col=1
)

# 4. create developer quality matrix plotting price vs amenities
# aggregate metrics for significant developers
quality_matrix = df[df['Developer_Clean'].isin(significant_developers)].groupby('Developer_Clean').agg({
    'Price': 'mean',
    'Amenity_Score': 'mean',
    'Developer_Clean': 'count'
}).round(1)
quality_matrix.columns = ['Avg_Price', 'Avg_Amenity_Score', 'Properties']

# select top 12 developers for clarity
top_quality_developers = quality_matrix.head(12)

# create scatter plot with bubble sizes based on portfolio size
fig.add_trace(
    go.Scatter(
        x=top_quality_developers['Avg_Amenity_Score'],
        y=top_quality_developers['Avg_Price'] / 100000,  # convert to lakhs
        mode='markers+text',
        text=[dev[:10] for dev in top_quality_developers.index],  # truncate long names
        textposition='top center',
        marker=dict(
            size=np.sqrt(top_quality_developers['Properties']) * 3,  # bubble size based on portfolio
            color=top_quality_developers['Avg_Price'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Avg Price", x=1.05)
        ),
        name='Developer Quality',
        hovertemplate='<b>%{text}</b><br>Amenity Score: %{round(x,1)}%<br>Avg Price: ₹%{round(y,1)}L<br><extra></extra>'
    ),
    row=2, col=2
)

# configure overall layout
fig.update_layout(
    title_text="Developer Premium Positioning Analysis",
    height=1200,
    showlegend=True,
    title_x=0.5
)

# set appropriate axis labels for each subplot
fig.update_yaxes(title_text="Property Count", row=1, col=1)
fig.update_yaxes(title_text="Average Price (Rs. Crores)", row=1, col=2)
fig.update_yaxes(title_text="Price per Sqft (Rs.)", row=2, col=1)
fig.update_xaxes(title_text="Average Amenity Score (%)", row=2, col=2)
fig.update_yaxes(title_text="Average Price (Rs. Lakhs)", row=2, col=2)

# rotate x-axis labels for better readability
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=1, col=2)

In [ ]:
"""Create developer amenities impact analysis"""

# select top developers and key amenities for analysis
top_developers = df['Developer_Clean'].value_counts().head(10).index
key_amenities = available_amenities[:8] if len(available_amenities) >= 8 else available_amenities

# create subplot layout for comprehensive developer analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Developer Amenity Provision Heatmap',
        'Top Developers by Amenity Score',
        'Developer Amenity Consistency',
        'Amenity Score vs Price Correlation'
    ),
    specs=[[{"type": "bar"}, {"type": "bar"}],
            [{"type": "bar"}, {"type": "scatter"}]]
)

# 1. analyze how well each developer provides different amenities
if key_amenities:
    # calculate what percentage of properties each developer includes specific amenities
    amenity_provision = {}
    for developer in top_developers[:6]:  # limit to top 6 for better visibility
        dev_data = df[df['Developer_Clean'] == developer]
        provision_rates = {}
        
        # check provision rate for each key amenity
        for amenity in key_amenities[:4]:  # focus on top 4 amenities for clarity
            if amenity in df.columns:
                # calculate percentage of properties that have this amenity
                rate = (dev_data[amenity].sum() / len(dev_data)) * 100
                provision_rates[amenity] = rate
        
        amenity_provision[developer] = provision_rates
    
    # create grouped bar chart showing amenity provision rates
    for amenity in key_amenities[:4]:
        amenity_values = [amenity_provision.get(dev, {}).get(amenity, 0) for dev in top_developers[:6]]
        
        fig.add_trace(
            go.Bar(
                name=amenity,
                x=[dev[:15] for dev in top_developers[:6]],  # truncate long developer names
                y=amenity_values,
                opacity=0.8
            ),
            row=1, col=1
        )

# 2. rank developers by their average amenity scores
# focus on developers with significant number of properties
developer_amenity_stats = df[df['Developer_Clean'].isin(significant_developers)].groupby('Developer_Clean').agg({
    'Amenity_Score': 'mean',
    'Total_Amenities': 'mean',
    'Developer_Clean': 'count'
}).round(1)
developer_amenity_stats.columns = ['Avg_Amenity_Score', 'Avg_Total_Amenities', 'Properties']

# get top 10 developers by amenity score
top_amenity_developers = developer_amenity_stats.nlargest(10, 'Avg_Amenity_Score')

fig.add_trace(
    go.Bar(
        x=[dev[:15] for dev in top_amenity_developers.index],  # truncate names for readability
        y=top_amenity_developers['Avg_Amenity_Score'],
        text=[f'{round(val,1)}%' for val in top_amenity_developers['Avg_Amenity_Score']],
        textposition='auto',
        marker_color='darkgreen',
        opacity=0.8,
        name='Amenity Score'
    ),
    row=1, col=2
)

# 3. measure how consistent each developer is with their amenity offerings
consistency_data = {}
for developer in top_developers[:8]:
    dev_data = df[df['Developer_Clean'] == developer]
    if len(dev_data) >= 3:  # need minimum properties for meaningful analysis
        # lower standard deviation means more consistent amenity provision
        amenity_std = dev_data['Amenity_Score'].std()
        avg_amenities = dev_data['Amenity_Score'].mean()
        # convert to consistency score where higher = more consistent
        consistency_score = max(0, 100 - amenity_std)
        consistency_data[developer] = consistency_score

# display consistency scores if we have data
if consistency_data:
    sorted_consistency = sorted(consistency_data.items(), key=lambda x: x[1], reverse=True)
    
    fig.add_trace(
        go.Bar(
            x=[dev[:15] for dev, score in sorted_consistency],
            y=[score for dev, score in sorted_consistency],
            text=[f'{round(score, 1)}' for dev, score in sorted_consistency],
            textposition='auto',
            marker_color='teal',
            opacity=0.8,
            name='Consistency Score'
        ),
        row=2, col=1
    )

# 4. explore relationship between amenity scores and pricing for each developer
for i, developer in enumerate(top_developers[:8]):
    dev_data = df[df['Developer_Clean'] == developer]
    
    if len(dev_data) >= 3:  # only include developers with enough data points
        fig.add_trace(
            go.Scatter(
                x=dev_data['Amenity_Score'],
                y=dev_data['Price'],
                mode='markers',
                name=developer[:12],  # shorter names for legend
                marker=dict(
                    size=6,
                    opacity=0.7
                ),
                showlegend=False,  # too many developers would clutter legend
                hovertemplate=f'<b>{developer}</b><br>Amenity Score: %{{round(x,1)}}%<br>Price: Rs.%{{round(y,1)}}<extra></extra>'
            ),
            row=2, col=2
        )

# configure overall plot appearance
fig.update_layout(
    title_text="Developer Amenities & Features Analysis",
    height=1200,
    showlegend=True,
    title_x=0.5
)

# set appropriate axis labels for each subplot
fig.update_yaxes(title_text="Provision Rate (%)", row=1, col=1)
fig.update_yaxes(title_text="Average Amenity Score (%)", row=1, col=2)
fig.update_yaxes(title_text="Consistency Score", row=2, col=1)
fig.update_xaxes(title_text="Amenity Score (%)", row=2, col=2)
fig.update_yaxes(title_text="Price (Rs.)", row=2, col=2)

# rotate x-axis labels to prevent overlap
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=1, col=2)
fig.update_xaxes(tickangle=45, row=2, col=1)